In [1]:
import numpy as np
import cv2
import pandas as pd
import os
from rdlib2 import *


def main():
    global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect,angle
    df = pd.read_excel("自動計測データ.xlsx")
    
    for radish in range(len(df)):
        
        def initcondition():
            global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect
            rect = (0,0,1,1)
            anglemode = False         # 角度モードオン
            framing = False           # 選択枠設定中
            framed = False       # 枠設定は完了している
            drawing = False
            angling = False
            angled = False
            lx,ly,dx,dy =0,0,0,0
            angle = 0
        
        def imgrotation(angle,center,img_src):
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
            size=tuple([img_src.shape[1], img_src.shape[0]])
            mg_rot = cv2.warpAffine(img_src, rotation_matrix, size, flags=cv2.INTER_CUBIC)
        
        
        initcondition()
        idata = df.iloc[radish]
        topdir = idata['topdir']  #  画像ファイルのパスのベース
        subdir = idata['subdir']  #  サブディレクトリ
        filename = idata['filename'] #  ファイル名
        check = idata['処理対象'] #  処理対象かどうかのフラグ　　test がTrueの時のみ意味がある
        if check !=10 : #  check が 10 でない画像はスルーする
                continue

        path = os.path.join(topdir,subdir,filename)
        print("処理対象画像 {}\n".format(path))
        src= cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            
        rdimg = getstandardShape(src, unitSize=UNIT, thres=0.25, setrotation = 0, showResult=False)
        _img,contours,hierarchy = cv2.findContours(rdimg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        rdcnt = contours[0] # 輪郭線情報　global 変数　
        
        x0,y0,width,height = cv2.boundingRect(rdcnt) # 外接矩形

        color = cv2.cvtColor(rdimg, cv2.COLOR_GRAY2BGR)

        mu = cv2.moments(rdimg, False)
        gx,gy= int(mu["m10"]/mu["m00"]) , int(mu["m01"]/mu["m00"])
      
        wname = "Angles"

        while True:
                img = color.copy()
                rect = (0,0,img.shape[1],img.shape[0])
                cv2.namedWindow(wname)
                cv2.setMouseCallback(wname, onMouse, [wname,img,gx,gy])
                cv2.imshow(wname, color)
                key = cv2.waitKey(0)
                if key==ord('r'):
                    initcondition()
                    continue
                elif key==13: # Enter で確定　次へ
                    print(head,bottom,angle,x0,height)
                    df.loc[radish,'CCUT0']=int(100*(head-y0)/height) if int(100*(head-y0)/height) > 0 else 3
                    df.loc[radish,'TCUT0']=int(100*(bottom-y0)/height) if int(100*(bottom-y0)/height) < 100 else 98
                    df.loc[radish,'ROT']=int(angle)
                    df.loc[radish,'処理対象']=1
                    df.to_excel('自動計測データ.xlsx', index=True, header=True)
                    break
                elif key==27: # ESC で終了
                    break
        if key==27: # ESC で終了
                cv2.destroyAllWindows()
                break
        
    cv2.waitKey(1)
    
lx,ly,dx,dy =0,0,0,0
def onMouse(event, x, y, flag, params):
    global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect,lx,ly,dx,dy
    wname, img, gx,gy= params
    if framed == False:
        if event == cv2.EVENT_LBUTTONDOWN:
            framing = True # 矩形描画モードオン
            lx,ly = x,y
            print("矩形描画スタート")
        elif event == cv2.EVENT_MOUSEMOVE: 
            img = color.copy()
            h, w = rdimg.shape[0], rdimg.shape[1]
            cv2.line(img, (x,0),(x,h-1),(255, 0, 0))
            cv2.line(img, (0, y), (w - 1, y), (255, 0, 0))       
            if framing == True :                      
                # img = color.copy()
                cv2.rectangle(img,(lx,ly),(x,y),(0,0,255),2)
                rect = (min(lx,x),min(ly,y),abs(lx-x),abs(ly-y))  
        elif event == cv2.EVENT_LBUTTONUP:
            if framing == True:
                framing = False
                framed = True
                print("フレーム一時確定")
                cv2.rectangle(img,(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,0,255),2)
                angling = True
                head = rect[1]
                bottom = rect[1]+rect[3]

    elif angled == False:
        if event == cv2.EVENT_LBUTTONUP:
                if angling == True:
                    dx = gx - x
                    dy = gy - y
                    angled = True 
                    print("角度確定",dx,dy)
                    cv2.rectangle(img,(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,0,255),2)
                    cv2.circle(img,(gx,gy), 3, (255,0,0), 3)
                    if x!=gx:
                        cv2.line(img, (x-500, y-int(500*(y-gy)/(x-gx))), (gx+500, gy+int(500*(y-gy)/(x-gx))), (255, 0, 0))
                    if dx == 0 :
                        angle = 0        
                    else:
                        if dx > 0:
                            angle = 180*np.arctan(dy/dx)/np.pi-90
                        else:
                            angle = 180*np.arctan(dy/dx)/np.pi+90
        elif event == cv2.EVENT_MOUSEMOVE:
                if angling == True : 
                    img = color.copy()
                    h, w = img.shape[0], img.shape[1]
                    if x!=gx:
                        cv2.line(img, (x-500, y-int(500*(y-gy)/(x-gx))), (gx+500, gy+int(500*(y-gy)/(x-gx))), (255, 0, 0))
                    cv2.line(img, (0, y), (w - 1, y), (255, 0, 0))
                    cv2.circle(img, (gx,gy), 3, (255,0,0), 3)
                    cv2.circle(img, (x,y), 3, (255,0,0), 3)   
        
    cv2.imshow(wname, img)
        
main()

処理対象画像 シルエット/17Conv_tri/17fuyuji1o08_l.jpg

矩形描画スタート
フレーム一時確定


# 上はフレーミングが先で画像固定、下は回転が先で、画像を回転させる

ただ、下は不安定

In [8]:
import numpy as np
import cv2
import pandas as pd
import os
from rdlib2 import *

def main():
    global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect,angle,color2
    df = pd.read_excel("自動計測データ.xlsx")
    
    for radish in range(len(df)):
        
        def initcondition():
            global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect
            rect = (0,0,1,1)
            anglemode = False         # 角度モードオン
            framing = False           # 選択枠設定中
            framed = False       # 枠設定は完了している
            drawing = False
            angling = True
            angled = False
            lx,ly,dx,dy =0,0,0,0
            angle = 0
            
        
        initcondition()
        idata = df.iloc[radish]
        topdir = idata['topdir']  #  画像ファイルのパスのベース
        subdir = idata['subdir']  #  サブディレクトリ
        filename = idata['filename'] #  ファイル名
        check = idata['処理対象'] #  処理対象かどうかのフラグ　　test がTrueの時のみ意味がある
        if check !=10 : #  check が 10 でない画像はスルーする
                continue

        path = os.path.join(topdir,subdir,filename)
        print("処理対象画像 {}\n".format(path))
        src= cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            
        rdimg = getstandardShape(src, unitSize=UNIT, thres=0.25, setrotation = 0, showResult=False)
        _img,contours,hierarchy = cv2.findContours(rdimg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        rdcnt = contours[0] # 輪郭線情報　global 変数　
        
        x0,y0,width,height = cv2.boundingRect(rdcnt) # 外接矩形

        color = cv2.cvtColor(rdimg, cv2.COLOR_GRAY2BGR)
        color2 = color.copy()

        mu = cv2.moments(rdimg, False)
        gx,gy= int(mu["m10"]/mu["m00"]) , int(mu["m01"]/mu["m00"])
      
        wname = "Angles"

        while True:
                img = color.copy()
                rect = (0,0,img.shape[1],img.shape[0])
                cv2.namedWindow(wname)
                cv2.setMouseCallback(wname, onMouse, [wname,img,gx,gy])
                cv2.imshow(wname, color)
                key = cv2.waitKey(0)
                if key==ord('r'):
                    initcondition()
                    color2=color.copy()
                    continue
                elif key==ord('b'):
                    framing == True
                    continue
                elif key==13: # Enter で確定　次へ
                    print(head,bottom,angle,x0,height)
                    df.loc[radish,'CCUT0']=int(100*(head-y0)/height) if int(100*(head-y0)/height) > 0 else 3
                    df.loc[radish,'TCUT0']=int(100*(bottom-y0)/height) if int(100*(bottom-y0)/height) < 100 else 98
                    df.loc[radish,'ROT']=int(angle)
                    df.loc[radish,'処理対象']=1
                    df.to_excel('自動計測データ.xlsx', index=True, header=True)
                    break
                elif key==27: # ESC で終了
                    break
        if key==27: # ESC で終了
                cv2.destroyAllWindows()
                break
        
    cv2.waitKey(1)

def onMouse(event, x, y, flag, params):
    global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect,lx,ly,dx,dy,color2
    wname, img, gx,gy= params
    
    def imgrotation(angle,center,img_src):
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
        size=tuple([img_src.shape[1], img_src.shape[0]])
        mg_rot = cv2.warpAffine(img_src, rotation_matrix, size, flags=cv2.INTER_CUBIC)
        return mg_rot
        
    if angled == False:
        if event == cv2.EVENT_LBUTTONDOWN:
            angling == True
        elif event == cv2.EVENT_LBUTTONUP:
                if angling == True:
                    dx = gx - x
                    dy = gy - y
                    angled = True
                    angling = False
                    print("角度確定",dx,dy)
                    if dx == 0 :
                        angle = 0        
                    else:
                        if dx > 0:
                            angle = 180*np.arctan(dy/dx)/np.pi-90
                        else:
                            angle = 180*np.arctan(dy/dx)/np.pi+90
                    color2 = imgrotation(angle,(gx,gy),color.copy())
                    print("imgrotation")
        elif event == cv2.EVENT_MOUSEMOVE:
                if angling == True : 
                    img = color2.copy()
                    h, w = img.shape[0], img.shape[1]
                    if x!=gx:
                        cv2.line(img, (x-500, y-int(500*(y-gy)/(x-gx))), (gx+500, gy+int(500*(y-gy)/(x-gx))), (255, 0, 0))
                    cv2.line(img, (0, y), (w - 1, y), (255, 0, 0))
                    cv2.circle(img, (gx,gy), 3, (255,0,0), 3)
                    cv2.circle(img, (x,y), 3, (255,0,0), 3)   
    elif framed == False: 
        if event == cv2.EVENT_LBUTTONDOWN:
            framing = True # 矩形描画モードオン
            lx,ly = x,y
            print("矩形描画スタート")
        elif event == cv2.EVENT_MOUSEMOVE: 
            img = color2.copy()
            h, w = rdimg.shape[0], rdimg.shape[1]
            cv2.line(img, (x,0),(x,h-1),(255, 0, 0))
            cv2.line(img, (0, y), (w - 1, y), (255, 0, 0))       
            if framing == True :                      
                # img = color.copy()
                cv2.rectangle(img,(lx,ly),(x,y),(0,0,255),2)
                rect = (min(lx,x),min(ly,y),abs(lx-x),abs(ly-y))  
        elif event == cv2.EVENT_LBUTTONUP:
            if framing == True:
                framing = False
                framed = True
                print("フレーム一時確定")
                cv2.rectangle(img,(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,0,255),2)
                head = rect[1]
                bottom = rect[1]+rect[3]
        
    cv2.imshow(wname, img)
        
main()

処理対象画像 シルエット/17Cylindric/17daruma3o01_l.jpg

角度確定 -26 161
imgrotation
矩形描画スタート
フレーム一時確定
78 325 9.173535442281917 154 253
処理対象画像 シルエット/17Cylindric/17daruma3o02_l.jpg

角度確定 -6 129
imgrotation
矩形描画スタート
フレーム一時確定
75 277 2.6630007660671424 163 217
処理対象画像 シルエット/17Cylindric/17daruma3o03_l.jpg

角度確定 14 155
imgrotation
矩形描画スタート
フレーム一時確定
79 321 -5.161098033360247 150 251
処理対象画像 シルエット/17Cylindric/17daruma3o05_l.jpg

角度確定 -4 131
imgrotation
矩形描画スタート
フレーム一時確定
77 289 1.7489460445542306 159 247
処理対象画像 シルエット/17Cylindric/17daruma3o09_l.jpg

角度確定 22 127
imgrotation
矩形描画スタート
フレーム一時確定
78 270 -9.827724011163113 154 224
処理対象画像 シルエット/17Cylindric/17daruma6o09_l.jpg

角度確定 12 140
imgrotation
矩形描画スタート
フレーム一時確定
79 287 -4.899092453787773 157 228
処理対象画像 シルエット/17Cylindric/17kohaku1o02_l.jpg

角度確定 -15 171
imgrotation
矩形描画スタート
フレーム一時確定
78 320 5.013113755035803 142 253
処理対象画像 シルエット/17Cylindric/17kohaku1o09_l.jpg

角度確定 -8 150
imgrotation
矩形描画スタート
フレーム一時確定
78 336 3.0528825147924294 147 253
処理対象画像 シルエット/17Cylindric/17kohak